# Simulating WFI Imaging Data with Roman I-Sim

***

## Server Information

**IMPORTANT: To run this tutorial, please make sure you are logged in the RRN with a medium or large server.** Running the parallelized examples in the advanced use cases will require a large server.

## Kernel Information and Read-Only Status

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

This notebook is read-only. You can run cells and make edits, but you must save changes to a different location. We recommend saving the notebook within your home directory, or to a new folder within your home (e.g. <span style="font-variant:small-caps;">file > save notebook as > my-nbs/nb.ipynb</span>). Note that a directory must exist before you attempt to add a notebook to it.

## Imports
 Libraries used
- *argparse* for formatting input options in romanisim
- *astroquery.gaia* for querying the Gaia catalog
- *astropy.coordinates* for storing celestial coordinates as Python objects
- *astropy.time* for storing time information as Python objects
- *astropy.table* for working with Astropy Table objects
- *galsim* for image simulations
- *numpy* for array operations
- *romanisim* for image simulations
- *s3fs* for accessing files in an S3 bucket

In the [Advanced Use Cases](#Advanced-Use-Cases) section further below, we include some additional imports:
- *pysiaf* for determining dither positions
- *dataclasses* for creating a class
- *typing* for doing type checking on inputs
- *dask* for parallelization

**Note:** `dask` is not installed in the Roman Calibration kernel by default. In the [Advanced Use Cases - Parallelized Simulations](#Parallelized-Simulations) section, the code cells have been commented out and there is a cell at the beginning of the section that will use `pip` to install the required packages.

In [ ]:
import argparse
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.table import vstack
import galsim
import numpy as np
from romanisim import gaia, bandpass, catalog, log, wcs, persistence, parameters, ris_make_utils as ris
import s3fs

## Introduction

The purpose of this notebook is to show how to generate simulated Level 1 (L1; uncalibrated ramp cubes) and Level 2 (L2; calibrated rate images) Roman Wide Field Instrument (WFI) Advanced Scientific Data Format (ASDF) files with [Roman I-Sim](https://romanisim.readthedocs.io/en/latest/) (package name `romanisim`). Details about the Roman data levels can be found in the [Data Levels and Products](https://roman-docs.stsci.edu/data-handbook-home/wfi-data-format/data-levels-and-products) article in the Roman Documentation System (RDox). Briefly, a L1 file contains a single uncalibrated ramp exposure in units of Data Numbers (DN).  L1 files are three-dimensional data cubes, one dimension for time and two dimensions for image coordinates, that are shaped as  arrays with (N resultants, 4096 image rows, 4096 image columns). A resultant is a sample up-the-ramp, and represents either a single read of the WFI detectors or multiple reads that have been combined. The L2 WFI data are calibrated images in instrumental units of DN / second.  They are two-dimensional arrays shaped as (4088 image rows, 4088 image columns).

***

## Tutorial Data

In this tutorial, we will create necessary data in memory or retrieve it from a catalog service. A catalog file is also available in the RRN S3 bucket, and can be streamed into memory using `astropy.table.Table` and the `s3fs` package instructions in the [Data Discovery and Access](../data_discovery_and_access/data_discovery_and_access.ipynb) tutorial. Also see the [RRN documentation](../../../markdown/simulated-data.md) for more information on the catalog available in the S3 bucket.

## Source Catalog Generation

The `romanisim` package offers two options for generating source catalogs:
1. Retrieve the source catalog from Gaia; or
2. Parametrically generate a catalog of stars and/or galaxies.

First, let's explore how to create a `romanisim`-compatible source catalog using Gaia. We will use a combination of `astroquery` and `romanisim` to query the Gaia catalog and then write the file in a format compatible with `romanisim`.

In the example below, we query the Gaia DR3 catalog for sources centered at (RA, Dec) = (270.94, -0.2) degrees within a radius of 1 degree.

**Note:** The Gaia query may take several minutes to complete.

In [ ]:
ra = 270.94  # Right ascension in degrees
dec = -0.2  # Declination in degrees
radius = 1  # Search radius in degrees

query = f'SELECT * FROM gaiadr3.gaia_source WHERE distance({ra}, {dec}, ra, dec) < {radius}'
job = Gaia.launch_job_async(query)
# print(job)
result = job.get_results()

Once we have the result from the Gaia query, we can transform it into a format compatible with Roman I-Sim:

In [ ]:
# Filter the Gaia results for stars and exclude bright stars
result = result[result['classprob_dsc_combmod_star'] >= 0.7]
result = result[result['phot_g_mean_mag'] > 16.5]

# Set the observation time
obs_time = '2026-10-31T00:00:00'

# Make the Roman I-Sim formatted catalog
gaia_catalog = gaia.gaia2romanisimcat(result, Time(obs_time), fluxfields=set(bandpass.galsim2roman_bandpass.values()))

We have excluded very bright stars (Gaia g-band > 16.5 mag in this example) because Roman I-Sim uses a maximum size of the simulated point spread function (PSF) in pixels that can result in the appearance of box-shaped boundaries around bright sources. This will be fixed in a future update.

When using a real catalog like Gaia, it is essential to remove any entries with missing information. This can be achieved with the code in the cell below:

In [ ]:
# Reject anything with missing fluxes or positions
names = [f for f in gaia_catalog.dtype.names if f[0] == 'F']
names += ['ra', 'dec']

bad = np.zeros(len(gaia_catalog), dtype='bool')
for b in names:
      bad = ~np.isfinite(gaia_catalog[b])
      if hasattr(gaia_catalog[b], 'mask'):
           bad |= gaia_catalog[b].mask
      gaia_catalog = gaia_catalog[~bad]

Now that we have a catalog, let's take a look at it. The catalog in memory is an `astropy.table.Table` object with over 1e5 rows:

In [ ]:
gaia_catalog

Alternatively, we can generate a completely synthetic catalog of stars and galaxies using tools in Roman I-Sim (see parameters in the cell below). In this tutorial, we will simulate a galaxy catalog and merge it with the Gaia star catalog. This approach addresses the limitation of Gaia's catalog, which includes only relatively bright sources, by adding galaxies. At the same time, real Gaia point sources are necessary for the Roman calibration pipeline to match images to Gaia astrometry. 

Note that we can additionally simulate a star catalog if desired. This may be useful for including stars fainter than the Gaia magnitude limit, or when the Gaia astrometric step in RomanCal is not required.

Note that in the cell below, we have commented out the last line, which would save the catalog to disk as an Enhanced Character-Separated Values (ECSV) file. We do not need the catalog to be saved on disk for this tutorial, but you may optionally uncomment the line to save the file if you wish. The same file is available in the S3 bucket for any other tutorials that may need it. Having a saved version of the catalog removes the need to re-run the Gaia query above if you need to start over, but you will need to add code to read the catalog file below.

In [ ]:
# Galaxy catalog parameters

ra = 270.94  # Right ascension of the catalog center in degrees
dec = -0.2  # Declination of the catalog center in degrees
radius = 1.0  # Radius of the catalog in degrees
n_gal = 10_000  # Number of galaxies
faint_mag = 20  # Faint magnitude limit of simulated sources
hlight_radius = 0.3  # Half-light radius at the faint magnitude limit in units of arcseconds
optical_element = 'F062 F087 F106 F129 F146 F158 F184 F213'.split()  # List of optical elements to simulate
seed = 123  # Random number seed for reproducibility

# Create galaxy catalog
galaxy_cat = catalog.make_galaxies(SkyCoord(ra, dec, unit='deg'), n_gal, radius=radius, index=0.4, faintmag=faint_mag, 
                                   hlr_at_faintmag=hlight_radius, bandpasses=optical_element, rng=None, seed=seed)

# Merge the galaxy and Gaia catalogs
full_catalog = vstack([galaxy_cat, gaia_catalog])

# full_catalog.write('full_catalog.ecsv', format='ascii.ecsv', overwrite=True)

The following cell is commented out, but if uncommented will create a simulated star catalog.

In [ ]:
#n_star = 30_000  # Number of stars

#star_cat = catalog.make_stars(SkyCoord(ra, dec, unit='deg'), n_star, radius=radius, index=5/3., faintmag=faint_mag, 
#                              truncation_radius=None, bandpasses=optical_element, rng=None, seed=seed)

As before, we have commented out the line that will write this to disk, and instead have kept it in memory. Below, let's print out the catalog and take a look:

In [ ]:
full_catalog

We can see galaxies at the top of the stacked catalog (notice type == "SER" for Sersic and values of n (the Sersic index) are not -1, while stars have type == PSF).

### Image Simulation

Here we show how to run the actual simulation using Roman I-Sim. The method for running the simulation for both L1 and L2 data is the same, so we will show an example for L2, and give instructions of how to modify this for L1.

In our example, we are simulating only a single image, so we have set the persistance to the default. Future updates may include how to simulate persistance from multiple exposures.

**Notes:** 

- Roman I-Sim allows the user to either use reference files from CRDS or to use no reference files. The latter is not recommended.
- Each detector is simulated separately. Below, in the [Advanced Use Cases - Parallelized Simulations](#Parallelized-Simulations) section, we include instructions on how to parallelize the simulations using Dask.
- Currently, the simulator does not include the effect of 1/f noise.
- Multi-accumulation (MA) tables control the total exposure time and sampling up-the-ramp. For more information, see the [MA table article](https://roman-docs.stsci.edu/raug/astronomers-proposal-tool-apt/appendix/appendix-wfi-multiaccum-tables) in the Roman APT Users Guide.

In this case, we will create an observation using the detector WFI01 and the F106 optical element. The observation is simulated to occur at UTC time 2026-10-31T00:00:00 and an exposure time of approximately 60 seconds.

**Note:** It will take several minutes to download the appropriate calibration reference files the first time this cell is run. Any changes to the settings below may require downloading additional files, which could increase the run time.

In [ ]:
obs_date = '2026-10-31T00:00:00'  # Datetime of the simulated exposure
sca = 1  # Change this number to simulate different WFI detectors 1 - 18
optical_element = 'F106'  # Optical element to simulate
ma_table_number = 3  # Multi-accumulation (MA) table number...do not recommend to change this as it must match files in CRDS
seed = 7  # Galsim random number generator seed for reproducibility
level = 2  # WFI data level to simulate...1 or 2
cal_level = 'cal' if level == 2 else 'uncal'  # File name extension for data calibration level
filename = f'r0003201001001001004_0001_wfi{sca:02d}_{optical_element.lower()}_{cal_level}.asdf'  # Output file name on disk. Only change the first part up to _WFI to change the rootname of the file.

# Set other arguments for use in Roman I-Sim. The code expects a specific format for these, so this is a little complicated looking.
parser = argparse.ArgumentParser()
parser.set_defaults(usecrds=True, webbpsf=True, level=level, filename=filename, drop_extra_dq=True)
args = parser.parse_args([])

# Set reference files to None for CRDS
for k in parameters.reference_data:
    parameters.reference_data[k] = None

# Set Galsim RNG object
rng = galsim.UniformDeviate(seed)

# Set default persistance information
persist = persistence.Persistence()

# Set metadata
metadata = ris.set_metadata(date=obs_date, bandpass=optical_element, sca=sca, ma_table_number=ma_table_number)

# Update the WCS info
wcs.fill_in_parameters(metadata, SkyCoord(ra, dec, unit='deg', frame='icrs'), boresight=False, pa_aper=0.0)

# Run the simulation
ris.simulate_image_file(args, metadata, full_catalog, rng, persist)

If we want to simulate an L1 ramp cube, then we can change the level variable above to 1, which will also change the output file name to `*_uncal.asdf`. The rest of the information stays the same.

## Advanced Use Cases

### Dithered Observations

Dithering is the process of shifting the telescope position slightly such that astronomical sources fall on different pixel positions in consecutive observations. Dithers comes in two flavors: 
- Large dithers: Used to fill the gaps between detectors and to reject pixels affected by undesirable effects
- Sub-pixel dithers: Used for sampling the point spread function (PSF)

If we want to create a set of dithered observations, we need to determine the new pointing of the WFI. Here we introduce a Python class that can take an initial right ascension, declination, and position angle of the WFI and then apply offsets to update those parameters for a new pointing. First, let's import some new packages and modules that will help, specifically:
- *pysiaf* for WFI coordinate transformations
- *dataclasses* for simplifying the definition of a class
- *typing* for type hinting of inputs and outputs

In this section, we show you how to determine the pointing positions necessary for dithering with the Roman WFI. In the [Advanced Use Cases - Simulating Dithered Observations in Parallel](#Simulating-Dithered-Observations-in-Parallel) section, we combine the dithering information and parallelization with Dask to show how to put this information to use.

In [ ]:
import pysiaf
from dataclasses import dataclass
from typing import Union

Next, we create a Python class called `PointWFI` that takes three inputs: ra, dec, and roll_angle. Defining a class may be a little complicated for those who are new to Python, so don't worry too much about the details for now. Just know that this class takes your input position, creates an attitude matrix for the spacecraft using PySIAF, applies the offsets, and updates the pointing information.

In [ ]:
@dataclass(init=True, repr=True)
class PointWFI:
    """
    Inputs
    ------
    ra (float): Right ascension of the target placed at the geometric 
                center of the Wide Field Instrument (WFI) focal plane
                array. This has units of degrees.
    dec (float): Declination of the target placed at the geometric
                 center of the WFI focal plane array. This has units
                 of degrees.
    roll_angle (float): Position angle of the WFI relative to the V3 axis
                        measured from North to East. A value of 0.0 degrees
                        would place the WFI in the "smiley face" orientation
                        (U-shaped) on the celestial sphere.

    Description
    -----------
    To use this class, insantiate it with your initial pointing like so:

        >>> point = PointWFI(ra=30, dec=-45, roll_angle=10)
    
    and then dither using the dither method:

        >>> point.dither(x_offset=10, y_offset=140)

    This would shift the WFI 10 arcseconds along the X-axis of the WFI
    and 140 arcseconds along the Y-axis of the WFI. These axes are in the ideal
    coordinate system of the WFI, i.e, with the WFI oriented in a U-shape with 
    +x to the right and +y up. You can pull the new pointing info out of the object 
    either as attributes or by just printing the object:

        >>> print(point.ra, point.dec)
        >>> 29.95536280064078 -44.977122003232786

    or

        >>> point
        >>> PointWFI(ra=29.95536280064078, dec=-44.977122003232786, roll_angle=10)
    """

    # Set default pointing parameters
    ra: float = 80.0
    dec: float = 30.0
    roll_angle: float = 0.0

    # Post init method sets some other defaults and initializes
    # the attitude matrix using PySIAF.
    def __post_init__(self) -> None:
        self.siaf_aperture = pysiaf.Siaf('Roman')['WFI_CEN']
        self.v2_ref = self.siaf_aperture.V2Ref
        self.v3_ref = self.siaf_aperture.V3Ref
        self.attitude_matrix = pysiaf.utils.rotations.attitude(self.v2_ref, self.v3_ref, self.ra,
                                        self.dec, self.roll_angle)
        self.siaf_aperture.set_attitude_matrix(self.attitude_matrix)

        # Compute the telescope pointing
        self._update_boresight_position()

    def _update_boresight_position(self):

        # Compute the telescope pointing based on the WFI target.
        boresight_sky = self.siaf_aperture.tel_to_sky(0, 0)
        self.tel_ra = boresight_sky[0]
        self.tel_dec = boresight_sky[1]
        self.attitude_matrix = pysiaf.utils.rotations.attitude(0, 0, self.tel_ra, self.tel_dec, self.roll_angle)
        self.tel_roll = pysiaf.utils.rotations.posangle(self.attitude_matrix, 0, 0)

    def dither(self, x_offset: Union[int, float],
               y_offset: Union[int, float]) -> None:
        """
        Purpose
        -------
        Take in an ideal X and Y offset in arcseconds and shift the telescope
        pointing to that position.

        Inputs
        ------
        x_offset (float): The offset in arcseconds in the ideal X direction.

        y_offset (float): The offset in arcseconds in the ideal Y direction.
        """

        # Compute the new RA and Dec. The WFI_CEN aperture is defined such
        # that the reference pixel position has ideal coodinates
        # (X, Y) = (0, 0).
        new_ra, new_dec = self.siaf_aperture.idl_to_sky(x_offset, y_offset)

        # Construct the new attitude matrix keeping the roll angle the same.
        self.attitude_matrix = pysiaf.utils.rotations.attitude(self.v2_ref, self.v3_ref, new_ra,
                                        new_dec, self.roll_angle)
        self.siaf_aperture.set_attitude_matrix(self.attitude_matrix)

        # Update the boresight position and the target position of the WFI.
        self._update_boresight_position()
        self.ra = new_ra
        self.dec = new_dec

Now let's dither the WFI. Dither patterns for the WFI are in development, but let's use this four-point box gap-filling pattern as an example. Note that the dither offsets are represented in the ideal X and Y directions (this means that +X is to the right and +Y is up when the WFI is in the U-shaped orientation with WFI07 and WFI16 in the upper-right and upper-left corners, respectively). The offsets are in units of arcseconds, and each offset represents the offset from the previous position. So, the zeroth position starts with offsets of 0.00 in both X and Y, and the first position is relative to the zeroth one, the second position is relative to the first, etc. Here is the pattern:

| Dither Step | Offset X (arcsec) | Offset Y (arcsec) |
| --- | --- | --- |
| 0 | 0.00 | 0.00 |
| 1 | 0.55 | 200.55 |
| 2 | 200.55 | 0.55 |
| 3 | 0.55 | -200.55 |

Now let's instantiate the `PointWFI` object with our initial pointing and move to the first dither position:

In [ ]:
pointing = PointWFI(ra=270.7, dec=-0.2, roll_angle=0.0)
pointing.dither(x_offset=0.55, y_offset=200.55)
print(pointing)

We can see that the WFI shifted slightly in both right ascension and declination, but not by 0.55 and 200.55 arcseconds. Remember that the WFI dither offsets are specified in a coordinate system local to the WFI, so the offsets on the sky will be different (hence the need for the class we created above). To make it easier to script our simulations, we can pull the variables out of the `pointing` object, such as below where we show how to retrieve the `.ra` attribute:

In [ ]:
pointing.ra

### Parallelized Simulations

Often, we will want to run a simulation using multiple detectors rather than just one at a time. Looping over the above in a serial fashion can take quite a long time, so we want to parallelize the work. In the example below, we will show how to parallelize the procedure with `Dask`. These cells are commented out by default, so to run them you need to uncomment all of the lines. Comments in code cells are marked with two # symbols (e.g., `## Comment`), so be sure to remove only the leading single # symbol.

In [ ]:
#!pip install dask[complete]

In [ ]:
#from dask.distributed import Client

At this point, it's helpful to redefine our simulation call above as a single function:

In [ ]:
#def run_romanisim(catalog, ra=80.0, dec=30.0, obs_date = '2026-10-31T00:00:00', sca=1, expnum=1, optical_element='F106', 
#                  ma_table_number=3, level=2, filename=f'r0003201001001001004', seed=5346):
#
#    cal_level = 'cal' if level == 2 else 'uncal'
#    filename = f'{filename}_{expnum:04d}_wfi{sca:02d}_{optical_element.lower()}_{cal_level}.asdf'
#
#    # Set other arguments for use in Roman I-Sim. The code expects a specific format for these, so this is a little complicated looking.
#    parser = argparse.ArgumentParser()
#    parser.set_defaults(usecrds=True, webbpsf=True, level=level, filename=filename, drop_extra_dq=True)
#    args = parser.parse_args([])
#
#    # Set reference files to None for CRDS
#    for k in parameters.reference_data:
#        parameters.reference_data[k] = None
#
#    # Set Galsim RNG object
#    rng = galsim.UniformDeviate(seed)
#
#    # Set default persistance information
#    persist = persistence.Persistence()
#
#    # Set metadata
#    metadata = ris.set_metadata(date=obs_date, bandpass=optical_element, sca=sca, ma_table_number=ma_table_number)
#
#    # Update the WCS info
#    wcs.fill_in_parameters(metadata, SkyCoord(ra, dec, unit='deg', frame='icrs'), boresight=False, pa_aper=0.0)
#
#    # Run the simulation
#    sim_result = ris.simulate_image_file(args, metadata, catalog, rng, persist)
#
#    # Clean up the memory
#    del sim_result

Now we initialize the Dask `Client()` and pass our simulation jobs to it. Dask will take care of scheduling the jobs and allocating resources. That said, we do have to be careful to not overload the Client. Set the number of WFI detectors to be simulated in the for loop appropriately to `n_detectors = 3` for a large RRN server.

The variable `offset` adds an offset to the numbering of the WFI detector names. If you want to simulate, e.g., detectors WFI01, WFI02, and WFI03, then set `n_detectors = 3` and `offset = 0`. If instead you want to simulate, e.g., detectors WFI04, WFI05, and WFI06, then set `n_detectors = 3` and `offset = 3`. The `expnum` variable lets you change the exposure number in the file name (this is useful if you are simulated a series of dithered observations).

**WARNING:** Please be cautious when parallelizing tasks such as Roman I-Sim as it can easily consume all of your RRN resources if handled incorrectly!

We have commented out the lines below. If you want to run the parallelized simulation, uncomment all of the lines in the following code cell.

**Note:** This cell may take several minutes to run. In addition, logging messages from `romanisim` and its dependencies will appear cluttered as they are executing simultaneously.

In [ ]:
## Number of detectors to simulate. Change offset to select
## higher detector IDs (e.g., n_detectors = 3 and offset = 3
## will simulate detectors WFI04, WFI05, and WFI06.
#n_detectors = 3
#offset = 0
#
## Change this to increment the exposure number in the simulation output filename
#expnum = 1
#
## Set up Dask client
## Give each simulation call its own worker so that no one worker exceeds
## the allocated memory.
#dask_client = Client(n_workers=n_detectors)
#
## Create simulation runs to send to the client
#tasks = []
#for i in range(n_detectors):
#
#    # Create simulations with the full_catalog defined above and for 3
#    # WFI detectors. Otherwise, use the default parameters.
#    tasks.append(dask_client.submit(run_romanisim, full_catalog, **{'ra': pointing.ra, 'dec': pointing.dec, 'sca': i+1+offset, 'expnum': expnum}))
#
## Wait for all tasks to complete
#results = dask_client.gather(tasks)
#
## Don't forget to close the Dask client!
#dask_client.close()

### Simulating Dithered Observations in Parallel

Now that we have demonstrated how to determine the WFI pointing positions necessary for any dither pattern, and how to parallelize Roman I-Sim simulations to simulate multiple detectors simultaneously, let's combine that information to simulate a series of dithered observations. As in the [Advanced Use Cases - Parallelized Simulations](#Parallelized-Simulations) section, we have commented out all of the code cells below. 

**Note:** If you have not reviewed the [Advanced Use Cases - Parallelized Simulations](#Parallelized-Simulations) section, please review the information in that part of the tutorial before proceeding. At a minimum, you will need to install Dask and define the Roman I-Sim wrapper function before running the cells below.

We will use the four-point box gap-filling dither pattern described in [Advanced Use Cases - Dithered Observations](#Dithered-Observations). We will also re-use the Roman I-Sim wrapper function that we defined for Dask. As you can see in the cell below, we are simulating three WFI detectors at a time at each dither position before proceeding to the next dither position. At each dither position, we incrementally update the exposure number. The resulting file names are: `r0003201001001001004_XXXX_wfiNN_f106_uncal.asdf`, where XXXX is the zero-padded exposure number (e.g., 0001) and NN is the WFI detector ID (e.g., WFI01). Note that these are L1 files rather than the L2 file that was simulated previously.

In [ ]:
## Initial pointing
#pointing = PointWFI(ra=270.94, dec=-0.2, roll_angle=0.0)
#
## Number of detectors to simulate. Change offset to select
## higher detector IDs (e.g., n_detectors = 3 and offset = 3
## will simulate detectors WFI04, WFI05, and WFI06.
#n_detectors = 3
#offset = 0
#
## Change this to increment the exposure number in the simulation output filename
#expnum = 1
#
## Set up Dask client
## Give each simulation call its own worker so that no one worker exceeds
## the allocated memory.
#dask_client = Client(n_workers=n_detectors)
#
## Create simulation runs to send to the client
#tasks = []
#for i in range(n_detectors):
#
#    # Create simulations with the full_catalog defined above and for 3
#    # WFI detectors. Otherwise, use the default parameters.
#    tasks.append(dask_client.submit(run_romanisim, full_catalog, **{'ra': pointing.ra, 'dec': pointing.dec, 'sca': i+1+offset, 'expnum': expnum, 'level': 1}))
#
## Wait for all tasks to complete
#results = dask_client.gather(tasks)
#
## Exposure 2
## Dither +0.55 +200.55
#pointing.dither(x_offset=0.55, y_offset=200.55)
#expnum=2
#tasks = []
#for i in range(n_detectors):
#    tasks.append(dask_client.submit(run_romanisim, full_catalog, **{'ra': pointing.ra, 'dec': pointing.dec, 'sca': i+1+offset, 'expnum': expnum, 'level': 1}))
#
## Wait for all tasks to complete
#results = dask_client.gather(tasks)
#
## Exposure 3
## Dither +200.55 +0.55
#pointing.dither(x_offset=200.55, y_offset=0.55)
#expnum=3
#tasks = []
#for i in range(n_detectors):
#    tasks.append(dask_client.submit(run_romanisim, full_catalog, **{'ra': pointing.ra, 'dec': pointing.dec, 'sca': i+1+offset, 'expnum': expnum, 'level': 1}))
#
## Wait for all tasks to complete
#results = dask_client.gather(tasks)
#
## Exposure 4
## Dither +0.55 -200.55
#pointing.dither(x_offset=0.55, y_offset=-200.55)
#expnum=4
#tasks = []
#for i in range(n_detectors):
#    tasks.append(dask_client.submit(run_romanisim, full_catalog, **{'ra': pointing.ra, 'dec': pointing.dec, 'sca': i+1+offset, 'expnum': expnum, 'level': 1}))
#    
## Wait for all tasks to complete
#results = dask_client.gather(tasks)
#
## Don't forget to close the Dask client!
#dask_client.close()

## Additional Resources
- [Roman I-Sim Documentation](https://romanisim.readthedocs.io/en/latest/index.html)
- [RomanCal Documentation](https://roman-pipeline.readthedocs.io/en/latest/index.html)
- [Roman Documentation System (RDox)](https://roman-docs.stsci.edu)

## About this Notebook
**Author:** Sanjib Sharma, Tyler Desjardins  
**Updated On:** 2025-01-10

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 